In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import geohash
import random
import math
import time
import pytz
from datetime import datetime

In [2]:
# # references
# # https://github.com/nlintz/TensorFlow-Tutorials
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist_softmax.py
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/fully_connected_feed.py
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist.py    
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/estimators/abalone.py
# https://github.com/lyft/challenge/blob/master/data_scientist/travel_time/description/travel_time.pdf
# linear regression https://aqibsaeed.github.io/2016-07-07-TensorflowLR/
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/linear_regression.py

In [3]:
# raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'],
#         'last_name': ['Miller', 'Jacobson', ".", 'Milner', 'Cooze'],
#         'age': [42, 52, 36, 24, 73],
#         'preTestScore': [4, 24, 31, ".", "."],
#         'postTestScore': ["25,000", "94,000", 57, 62, 70]}
# df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
# df
inputdf = pd.read_csv("train.csv")
df = inputdf

In [7]:
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT

def transform_dataset(df):
    def get_geohash_from_concat_latlon(latlon):
        lat, lon = latlon.split(":")
        lat = float(lat)
        lon = float(lon)        
        precision = 7    
        if (lat < -1000.0 or lat > 1000.0 or lon < -1000.0 or lon > 1000.0):
            return 0
        return geohash.encode(lat, lon, precision)

    tz = pytz.timezone('America/New_York')
    def get_local_time(timestamp):
        d = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.utc)
        dt = d.astimezone(tz)
        return dt

    def get_start_of_the_day(start_timestamp):
        return start_timestamp.replace(hour=0, minute=0, second=0)

    
#     d["geo"] = d["start_geohash"] + ":" + d["end_geohash"]
    start_geo_1 = df["start_lat"].astype(str) + ":" + df["start_lng"].astype(str)
    start_geo = start_geo_1.apply(get_geohash_from_concat_latlon)
    start_geo.name = 'start_geohash'
    df = pd.concat([df, start_geo], axis = 1)    
    del start_geo_1

    end_geo_1 = df["end_lat"].astype(str) + ":" + df["end_lng"].astype(str)
    end_geo = end_geo_1.apply(get_geohash_from_concat_latlon)
    end_geo.name = 'end_geohash'
    df = pd.concat([df, end_geo], axis = 1)    
    del end_geo_1
    
    start_time = df["start_timestamp"].apply(get_local_time)
    start_time.name = 'start_timestamp_obj'
    df = pd.concat([df, start_time], axis = 1)    
    # start_time[:10]

    start = df["start_timestamp_obj"].apply(get_start_of_the_day)
    start.name = 'start_of_day'
    df = pd.concat([df, start], axis = 1)

    hour = df["start_timestamp_obj"].dt.hour
    hour.name = 'hour'
    df = pd.concat([df, hour], axis = 1)

    minute = df["start_timestamp_obj"].dt.minute
    minute.name = 'minute'
    df = pd.concat([df, minute], axis = 1)

    weekday = df["start_timestamp_obj"].dt.weekday
    weekday.name = 'weekday'
    df = pd.concat([df, weekday], axis = 1)

    df["geo"] = df["start_geohash"] + ":" + df["end_geohash"]
    del start_time
    del end_geo
    del hour
    del minute
    del weekday
    outdf = df.copy()
    outdf.drop('start_lng', axis=1, inplace=True)
    outdf.drop('start_lat', axis=1, inplace=True)
    outdf.drop('end_lng', axis=1, inplace=True)
    outdf.drop('end_lat', axis=1, inplace=True)
    outdf.drop('start_timestamp', axis=1, inplace=True)
    outdf.drop('start_timestamp_obj', axis=1, inplace=True)
    outdf.drop('start_of_day', axis=1, inplace=True)
    return outdf
#     df = pd.concat([df, pd.DataFrame(start_time, index=df.index)], axis = 1)
dfff = pd.read_csv("test.csv")
d = dfff[:100].copy()
d = transform_dataset(d)
d[:10]

,row_id,start_geohash,end_geohash,hour,minute,weekday,geo
0,0,dr5rsjx,dr5rspd,3,13,6,dr5rsjx:dr5rspd
1,1,dr5rusx,dr5ruy5,13,2,1,dr5rusx:dr5ruy5
2,2,dr5rvjh,dr5rvnm,13,2,1,dr5rvjh:dr5rvnm
3,3,dr5ru2k,dr72jhr,4,8,3,dr5ru2k:dr72jhr
4,4,dr5rug9,dr5ruqp,23,9,2,dr5rug9:dr5ruqp
5,5,dr5ru2k,dr5rst4,20,41,3,dr5ru2k:dr5rst4
6,6,dr5ruuv,dr5ru0v,22,37,5,dr5ruuv:dr5ru0v
7,7,dr5ru14,dr5ru53,11,58,5,dr5ru14:dr5ru53
8,8,dr5rvj5,dr5ru6d,23,22,2,dr5rvj5:dr5ru6d
9,9,dr5rus3,dr5ruv0,13,56,3,dr5rus3:dr5ruv0


In [8]:
df = transform_dataset(df)

In [9]:
# df.to_csv("train_mod.csv", index=False)
df[:1]

,row_id,duration,start_geohash,end_geohash,hour,minute,weekday,geo
0,0,112,dr5regs,dr5reuz,23,33,5,dr5regs:dr5reuz


In [ ]:
# from collections import defaultdict
# g = set()
# dictionary = {}
# reverse_dictionary = {}
# for index, row in df.iterrows():
#     g.add(row['start_geohash'])
#     g.add(row['end_geohash'])
# for i, item in enumerate(g):
#     dictionary[i] = item
#     reverse_dictionary[item] = i

# tup = defaultdict(list)
# for index, row in df.iterrows():
#     tup[row['start_geohash']].append((row['duration'], row['end_geohash']))
# glist = list(g)
    
# #     gmap[row['start_geohash']]

In [10]:
from collections import defaultdict
dictionary = {}
reverse_dictionary = {}
start_list = list(df['start_geohash'])
end_list = list(df['end_geohash'])
geoset = set(start_list + end_list)

for i, item in enumerate(geoset):
    dictionary[i] = item
    reverse_dictionary[item] = i


In [13]:
tup = defaultdict(list)
def geo_mapping(combo):
    start, end, duration = combo.split(":")
    tup[start].append((int(duration), end))
    
# df["geo"] = df["start_geohash"] + ":" + df["end_geohash"] + ":" + df["duration"].astype(str)
_ = df["geo"].apply(geo_mapping)


In [14]:
sorted_tup = {}
for k, v in tup.iteritems():
    sorted_tup[k] = sorted(tup[k])[:300]


In [15]:
def dict_head(d, num=10):
    i = 0
    for k, v in d.iteritems():
        print k, v
        i += 1
        if i > num:
            break


In [16]:
# dict_head(reverse_dictionary)
dict_head(reverse_dictionary)

dr5x2fk 43101
dr727j3 22636
dr724d9 1
dr724d3 3
dr724d7 4
dr5x9zz 5
dr5x9zy 6
dr5x9zw 7
dr5x9zv 8
dr5x9zu 9
dr5x9zt 10


In [17]:
data_index = 0

In [18]:
### BEGINNING OF THE EMBEDDINGS MODEL

def generate_batch(batch_size):
    global data_index
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    dictionary[data_index]
    i = 0
    while i < batch_size:
        item = data_index
        element = sorted_tup.get(dictionary[data_index])
        if not element:
            data_index = (data_index + 1) % len(sorted_tup)
            continue
        options = sorted_tup[dictionary[data_index]]
        choice = random.choice(options[:300])
        batch[i] = item
        labels[i, 0] = reverse_dictionary[choice[1]]
        data_index = (data_index + 1) % len(sorted_tup)
        i += 1
    return batch, labels
batch, labels = generate_batch(10)
print 'batch', batch
print 'labels', labels

batch [ 0  2  4  5  6  9 15 21 23 34]
labels [[    0]
 [ 5133]
 [    4]
 [15182]
 [20409]
 [39681]
 [   15]
 [ 6454]
 [23540]
 [ 8020]]


In [19]:
batch_size = 64
embedding_size = 64  # Dimension of the embedding vector.
num_sampled = 64
vocabulary_size = len(geoset)

graph = tf.Graph()

with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int64, shape=[batch_size])
    train_labels = tf.placeholder(tf.int64, shape=[batch_size, 1])

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
#     valid_embeddings = tf.nn.embedding_lookup(
#       normalized_embeddings, valid_dataset)
#     similarity = tf.matmul(
#       valid_embeddings, normalized_embeddings, transpose_b=True)
    init = tf.global_variables_initializer()


In [20]:
# Step 5: Begin training.
num_steps = 200000
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

    final_embeddings = normalized_embeddings.eval()


Initialized
('Average loss at step ', 0, ': ', 276.7401123046875)
('Average loss at step ', 2000, ': ', 152.06010083770752)
('Average loss at step ', 4000, ': ', 72.915577672481533)
('Average loss at step ', 6000, ': ', 33.382292408347126)
('Average loss at step ', 8000, ': ', 18.646306135296822)
('Average loss at step ', 10000, ': ', 12.102113711416722)
('Average loss at step ', 12000, ': ', 7.9083063614815474)
('Average loss at step ', 14000, ': ', 5.6802085246592764)
('Average loss at step ', 16000, ': ', 3.8197205985933542)
('Average loss at step ', 18000, ': ', 3.0549071153029801)
('Average loss at step ', 20000, ': ', 2.4692361017763615)
('Average loss at step ', 22000, ': ', 1.8839523471817374)
('Average loss at step ', 24000, ': ', 1.5779076157510281)
('Average loss at step ', 26000, ': ', 1.3261253466084599)
('Average loss at step ', 28000, ': ', 1.1932069675326347)
('Average loss at step ', 30000, ': ', 1.068209701269865)
('Average loss at step ', 32000, ': ', 1.0161080236621

In [21]:
type(final_embeddings)

numpy.ndarray

In [22]:
input_batch = 32
data_size = vocabulary_size
training_size = vocabulary_size - vocabulary_size//10
testing_size = data_size - training_size
INPUT_FEATURES = 131
# start_timestamp	duration	start_geohash	end_geohash	start_timestamp_obj	start_of_day	weekday	hour	minute
""" Generate the input feature dataset """
def get_batch(batch_size, df):
    global batch_index
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    for i in range(batch_size):
        index = (batch_index + i) % training_size
        batch[i, 0] = df.iloc[index]["weekday"]
        batch[i, 1] = df.iloc[index]["hour"]
        batch[i, 2] = df.iloc[index]["minute"]
        start_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[index]["start_geohash"]]]
        end_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[index]["end_geohash"]]]
        embedding_start_index = 3
        for j in range(embedding_size):
            batch[i, j + embedding_start_index] = start_geohash_embedding[j]
        end_geohash_emebedding_start = embedding_start_index + embedding_size
        for j in range(embedding_size):
            batch[i, j + end_geohash_emebedding_start] = start_geohash_embedding[j]
        labels[i, 0] = df.iloc[index]["duration"]
    batch_index += batch_size % training_size
    return batch, labels

def get_test_batch(batch_size, df):
    indices = []
    for i in range(batch_size):
        indices.append(random.randrange(0, len(df)))
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.float32)
    for i, position in enumerate(indices):
        batch[i, 0] = df.iloc[position]["weekday"]
        batch[i, 1] = df.iloc[position]["hour"]
        batch[i, 2] = df.iloc[position]["minute"]
        start_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[position]["start_geohash"]]]
        end_geohash_embedding = final_embeddings[reverse_dictionary[df.iloc[position]["end_geohash"]]]
        embedding_start_index = 3
        for j in range(embedding_size):
            batch[i, j + embedding_start_index] = start_geohash_embedding[j]
        end_geohash_emebedding_start = embedding_start_index + embedding_size
        for j in range(embedding_size):
            batch[i, j + end_geohash_emebedding_start] = start_geohash_embedding[j]
        labels[i, 0] = df.iloc[position]["duration"]
    return batch, labels

def get_full_test_data(limit=None):
    path = "test.csv"
    df = pd.read_csv("test.csv")
    if limit:
        df = df[:limit]
    df = transform_dataset(df)
    batch_size = len(df)
    batch = np.ndarray(shape=(batch_size, INPUT_FEATURES), dtype=np.float32)
    for i in range(batch_size):
        batch[i, 0] = df.iloc[i]["weekday"]
        batch[i, 1] = df.iloc[i]["hour"]
        batch[i, 2] = df.iloc[i]["minute"]
        start_geohash_index = reverse_dictionary.get(df.iloc[i]["start_geohash"], random.randrange(0, 1000))
        end_geohash_index = reverse_dictionary.get(df.iloc[i]["end_geohash"], random.randrange(0, 1000))
        start_geohash_embedding = final_embeddings[start_geohash_index]
        end_geohash_embedding = final_embeddings[end_geohash_index]
        embedding_start_index = 3
        for j in range(embedding_size):
            batch[i, j + embedding_start_index] = start_geohash_embedding[j]
        end_geohash_emebedding_start = embedding_start_index + embedding_size
        for j in range(embedding_size):
            batch[i, j + end_geohash_emebedding_start] = start_geohash_embedding[j]
    return batch


In [23]:
def input_placeholders():
    with tf.name_scope('input_layer'):
        features = tf.placeholder(tf.float32, shape=[None, INPUT_FEATURES])
        train_labels = tf.placeholder(tf.float32, shape=[None, 1])
    return features, train_labels
def inference(features, hidden1_units, hidden2_units):
    """ Building the tensorflow model """
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([INPUT_FEATURES, hidden1_units],
                                stddev=1.0 / math.sqrt(float(INPUT_FEATURES))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features, weights) + biases)
    # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, 1],
                                stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([1]),
                             name='biases')
        output_linear = tf.matmul(hidden2, weights) + biases
    return output_linear

In [24]:
def get_loss(output_linear, labels, n_samples):
    """Calculates the loss from the output_linear and the labels.
    Args:
      logits: output_linear tensor, float - [batch_size, 1].
      labels: Labels tensor, int32 - [batch_size].
    Returns:
      loss: Loss tensor of type float.
    """
    labels = tf.to_float(labels)
#     cost = tf.reduce_mean(tf.square(output_linear - labels))
    cost = tf.reduce_sum(tf.pow(output_linear-labels, 2))/(2*n_samples)
    return cost


In [25]:

def training(loss, starter_learning_rate, decay_steps):
    """Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
      loss: Loss tensor, from loss().
      learning_rate: The learning rate to use for gradient descent.
    Returns:
      train_op: The Op for training.
    """
    global_step = tf.Variable(0, name='global_step', trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               decay_steps, 0.96, staircase=True)   
    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', loss)
    # Create the gradient descent optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Create a variable to track the global step.
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


In [26]:
def get_evaluation(output_linear, labels):

    diff = tf.subtract(output_linear, labels)
    return tf.reduce_mean(diff)

In [ ]:
full_test_data = batch_data
# full_test_data = get_full_test_data()

In [ ]:
# data_size = vocabulary_size
# training_size = vocabulary_size - vocabulary_size//10
# testing_size = data_size - training_size
final_results = None
batch_index = 0
num_steps = 150000
input_batch = 4
starter_learning_rate = 0.001
decay_steps = 10000
def start_training(num_steps):
#     sess = tf.InteractiveSession()
#     tf.global_variables_initializer().run()
    with tf.Graph().as_default():
        features, labels = input_placeholders()
        linear_inference = inference(features, 128, 64)
        loss = get_loss(linear_inference, labels, training_size)
        train_op = training(loss, starter_learning_rate, decay_steps)
        evaluation = get_evaluation(linear_inference, labels)
        summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        sess = tf.Session()
        summary_writer = tf.summary.FileWriter("/tmp/rides/", sess.graph)
        sess.run(init)
        training_data = df[:training_size]
        test_data = df[testing_size + 1:]
        avg_loss = 0
        for step in xrange(num_steps):
            start_time = time.time()
            batch_data, label_data = get_batch(input_batch, training_data)
            feed_dict = {features:batch_data, labels:label_data}
            _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)
            duration = time.time() - start_time
            avg_loss += loss_value

          # Write the summaries and print an overview fairly often.
            if step % 3000 == 0:
                # Print status to stdout.
                print('Step %d: avg loss = %.2f (%.3f sec)' % (step, avg_loss/2000, duration))
                avg_loss = 0
                # Update the events file.
#                 summary_str = sess.run(summary, feed_dict=feed_dict)
#                 summary_writer.add_summary(summary_str, step)
#                 summary_writer.flush()
            if step % (num_steps - 1) == 0:
                test_batch_data, test_label_data = get_test_batch(20000, training_data)
                feed_dict = {features:test_batch_data, labels:test_label_data}
                results = sess.run(evaluation, feed_dict=feed_dict)
                print 'Mean difference in testing data : %s' % results
#         batch_data = get_full_test_data()
        feed_dict = {features:full_test_data}
        global final_results
        final_results = sess.run(linear_inference, feed_dict=feed_dict)
start_training(num_steps)


Step 0: avg loss = 0.01 (0.011 sec)
Mean difference in testing data : -736.751
Step 3000: avg loss = 37.45 (0.004 sec)
Step 6000: avg loss = 40.51 (0.004 sec)
Step 9000: avg loss = 25.49 (0.004 sec)
Step 12000: avg loss = 25.69 (0.004 sec)
Step 15000: avg loss = 35.50 (0.004 sec)
Step 18000: avg loss = 29.58 (0.004 sec)
Step 21000: avg loss = 23.86 (0.004 sec)
Step 24000: avg loss = 26.72 (0.004 sec)
Step 27000: avg loss = 38.24 (0.004 sec)
Step 30000: avg loss = 23.09 (0.004 sec)
Step 33000: avg loss = 25.29 (0.004 sec)
Step 36000: avg loss = 35.00 (0.004 sec)
Step 39000: avg loss = 28.68 (0.004 sec)
Step 42000: avg loss = 23.73 (0.004 sec)
Step 45000: avg loss = 35.20 (0.004 sec)
Step 48000: avg loss = 28.75 (0.004 sec)
Step 51000: avg loss = 23.59 (0.004 sec)
Step 54000: avg loss = 25.82 (0.004 sec)
Step 57000: avg loss = 37.77 (0.004 sec)
Step 60000: avg loss = 22.94 (0.004 sec)
Step 63000: avg loss = 24.37 (0.004 sec)
Step 66000: avg loss = 34.07 (0.004 sec)
Step 69000: avg loss =

In [273]:
final_results[:1]
final_output = pd.DataFrame({'duration':final_results[:,0]})

array([[ 250.06097412],
       [ 340.99884033],
       [ 337.78717041],
       [ 196.7807312 ],
       [ 639.41094971],
       [ 827.86187744],
       [ 868.50714111],
       [ 796.97949219],
       [ 741.89941406],
       [ 802.07250977]], dtype=float32)

In [ ]:
final_output[:1]
final_output.to_csv("results.csv", index=False)

In [ ]:
# https://gist.github.com/mchirico/bcc376fb336b73f24b29

In [261]:
def assert_geohashes_in_set(df, reverse_dictionary):
    for index, row in df.iterrows():
        missing = set()
        if not reverse_dictionary.get(row["start_geohash"]):
            missing.add(row["start_geohash"])
        if not reverse_dictionary.get(row["end_geohash"]):
            missing.add(row["end_geohash"])
        if len(missing) > 0:
            raise Exception("end_geohash is not in global dataset: %s"% len(missing))

def transform_input():
    df = pd.read_csv("test.csv")
    df['start_geohash'] = df.apply(geohash_encode_start, axis=1)
    print 'end geohash'
    df['end_geohash'] = df.apply(geohash_encode_end, axis=1)    
#     df["start_timestamp_obj"] = df.apply(get_local_time, axis=1)
#     df.to_csv("test_mod.csv")
#     df["start_of_day"] = df.apply(get_start_of_the_day, axis=1)
#     df["weekday"] = df.apply(get_weekday, axis=1)
#     df["hour"] = df.apply(get_hour, axis=1)
#     df["minute"] = df.apply(get_min, axis=1)
    return df

def handle_test_data():
    df = pd.read_csv("test.csv")
    df = transform_input(df)
    assert_geohashes_in_set(df, reverse_dictionary)
    return df

In [266]:
# dftest = transform_input()
# assert_geohashes_in_set(dftest, reverse_dictionary)
assert_geohashes_in_set(dftest, reverse_dictionary)

Exception: end_geohash is not in global dataset: 1